# Introduction
K-Means is an unsupervised ML algorithm used to group data points into a predefined number of clusters. The "K" in K-Means is the number of clusters that is a predefined, and the algorithm clusters the data points into the defined K number of clusters. Meaning, the data points are classified into K clusters.

A training dataset of d-dimensions along with the K value is fed as input to the algorithm. For each of the row in the training data, the algorithm computes which cluster the data points belong to.

# K-Means Clustering
Consider a 2D dataset with K = 3, the 3 clusters are namely $S_1$, $S_2$ and $S_3$ with 3 centroids $C_1$, $C_2$ and $C_3$. In an ideal case,
- $S_1 \cup S_2 \cup S_3 = \text{Universal Set} = U$, and
- $S_1 \cap S_2 \cap S_3 = \text{Null Set} = \phi$

The above indicates that all of the data points belong to one or the other set and no data point exists in more than one set or cluster.

Now consider a set of points $S_i$ of size |$S_i$| (cardinality of the set). The centroid of this set can be found by summing all the points in the set (x_j) and averaging them by the size of the set (|$S_i$|). Mathematically,

$C_i$ = $\frac{1}{|S_i|} \sum{x_j} \backepsilon x_j \in S_i$

The "sum of data points" refers to the element-wise addition of all the data points within a cluster. For example, if a cluster has 3 data points,
- `x1 = [1, 2, 3]`
- `x2 = [4, 5, 6]`
- `x3 = [7, 8, 9]`

then, the sum of data points would be: `x1 + x2 + x3 = [1+4+7, 2+5+8, 3+6+9] = [12, 15, 18]`.

The goal is to find the centroids, $C_1$, $C_2$ and $C_3$, ..., $C_k$.

### How are clusters created using centroids?
Distance of a point is measured from all of the centroids, the point is then assigned to the cluster whose distance is the least from the centroid of that cluster.

### Mathematical formulation of K-Means
Given a dataset $D$, the goal is to find the clusters $S_1$, $S_2$ and $S_3$, ..., $S_k$ and their centroids, $C_1$, $C_2$ and $C_3$, ..., $C_k$ such that, the inter cluster distance is maximum (clear separation) and intra cluster distance is minimum (tight clustering).

Applying gradient descent does not help because, the result of gradient descent can be a fraction, and this does not help the case of clustering as data points can either belong to a cluster or not as a fractional value would mean that a data point would partially belong to a cluster. Also, in case the interger values are used to solve the optimization problem, the solution would have NP hard complexity.

To deal with the complexity problem, optimization is done using approximation algorithms. Lloyd's Algorithm is one such approximation algorithm.

### Lloyd's Algorithm
1. Initialization: From the given dataset $D$, $K$ points are picked at random, and are assumed to be the centroids. Let these be denoted as $C_1$, $C_2$ and $C_3$, ..., $C_k$.
2. Assignment: For each data point $x_i$ in the dataset, the distance of the data point from each of the $K$ centroids is computed. The centroid at the nearest is picked, let this be denoted as $C_j$. The point $x_i$ is assigned to the centroid $S_j$ which is associated with the centroid $C_j$.
3. Recompute the Centroid (update step): Now that all the data points have been grouped into their respective clusters, the centroid for each and every cluster is updated. The centroids are update using the equation, $C_i$ = $\frac{1}{|S_i|} \sum{x_j} \backepsilon x_j \in S_i$.
4. This assignment of data points to the cluster and updating the cluster centroids is repeated until convergence. The term convergence here points to the scenario where the cluster centroids do not change much.